In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

gtimer = GlobalTimer.instance()
gtimer.reset()

In [2]:
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *
crob = CombinedRobot(connection_list=(False, False))

connection_list
(False, False)


In [3]:
GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
VISUALIZE = False

In [4]:
DATA_PATH = "./data"
try: os.mkdir(DATA_PATH)
except: pass

In [5]:
import datetime
def get_now():
    return datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [6]:
CURRENT_PATH = os.path.join(DATA_PATH,get_now())
os.mkdir(CURRENT_PATH)

## Global params definition

In [7]:
GLOBAL_PARAMS = dict(
    BASE_LINK = "base_link",
    MAX_REACH_DICT = {'indy0': 1.5, 'panda1': 1.5},
    REACH_OFFSET_DICT = {'indy0': (0,0,0.3), 'panda1': (0,0,0.3)},
    GRIPPER_REFS = {"indy0":{"link_name": "indy0_tcp", "tcp_ref": [0,0,0.14], "depth_range": (-0.07,0.00), "width_range": (0.03,0.05), "bname": "grip0"}, 
                    "panda1":{"link_name": "panda1_hand", "tcp_ref": [0,0,0.112], "depth_range": (-0.04,0.00), "width_range": (0.03,0.06), "bname": "grip1"}
                   },
    Nrbt = len(crob.robot_names), # 2
    Njoints = len(crob.joint_names), # 13
    WDH = (3,3,3),
    WDH_MIN_RBT = (0.75,0.75,0.5), 
    WDH_MAX_RBT = (2.25,2.25,1.0),
    L_CELL = 0.2,
    MIN_DIST_ROBOT = 1,
    No = 50,
    RATIO_COVER = 2.0,
    RATIO_DIMS = 2.0,
    LINK_COLL_MARGIN = 0.01,
    N_retry = 1
)
SAMPLED_DATA = defaultdict(dict)

In [8]:
save_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME), GLOBAL_PARAMS)

## set params

In [9]:
__local_params = locals()
for k,v in GLOBAL_PARAMS.items():
    __local_params[k] = v
CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH 
Nw, Nd, Nh = Nwdh = int(Ws/L_CELL), int(Ds/L_CELL), int(Hs/L_CELL)
L_MAX = L_CELL*RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw*Nd*Nh, Nw, Nd, Nh))

scene size: 3375 (15,15,15)


In [10]:
SAMPLED_DATA["WORLD"]["Trbt_dict"] = Trbt_dict= sample_Trbt(Nrbt, crob.robot_names, WDH_MIN_RBT, WDH_MAX_RBT, MIN_DIST_ROBOT)
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

In [11]:
WORLD_PATH = os.path.join(CURRENT_PATH, "WORLD-"+get_now())
os.mkdir(WORLD_PATH)
save_json(os.path.join(WORLD_PATH, WORLD_FILENAME), SAMPLED_DATA["WORLD"])

# LEVEL: WORLD

## put robot

In [12]:
cam = None
# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict, crob.custom_limits, start_rviz=VISUALIZE)
ghnd = GeometryHandle(urdf_content)
time.sleep(2)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


In [13]:
# set graph
if "graph" in locals():
    graph.clear_markers()
    graph.clear_highlight()
    graph.ghnd.clear()
    graph.__init__(ghnd=graph.ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                   urdf_content=urdf_content, combined_robot=crob)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    if VISUALIZE: graph.set_rviz()
else:
    graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                            urdf_content=urdf_content, combined_robot=crob)
    graph.set_camera(cam)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    if VISUALIZE: graph.set_rviz()
    
    # start UI
    ui_broker = UIBroker(graph)
    ui_broker.start_server()
    
    # set rviz
    if VISUALIZE: graph.set_rviz(crob.home_pose)
    ui_broker.set_tables()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


In [14]:
for gripper in GRIPPER_REFS.values():
    graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0,0,0], link_name=gripper['link_name'])
graph.register_binder(name='base', _type=PlaceFrame, point=[0,0,0], rpy=[0,0,0], link_name=BASE_LINK)
vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK, 
                                             dims=(0,0,0), center=(0,0,0), rpy=(0,0,0), collision=False, display=False
                                            )
graph.add_object("virtual",
                 SingleHandleObject(_object=vtem,
                                   action_point=FramePoint(name="point", _object=vtem, point=(0,0,0), rpy=(0,0,0), name_full=None)), 
                 binding=("point", "base"))

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## show workspace

In [15]:
# show_workspace(graph, Nwdh, CENTER, L_CELL)

In [16]:
# remove_geometries_by_prefix(graph, "workspace")
# remove_geometries_by_prefix(graph, "grid")

# LEVEL: PREMISE

## init joints

In [17]:
from pkg.data_collecting.sampling import *

In [18]:
SAMPLED_DATA["SCENE"]["Q_s"], _, _, _, _ = Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph)
SAMPLED_DATA["SCENE"]["Q_s_dict"] = Q_s_dict = list2dict(Q_s, crob.joint_names)
        
if VISUALIZE: 
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)

## extract available cells

In [19]:
gtimer.tic("extract_available")
# coll_boxes = get_colliding_cells(Nwdh, L_CELL, link_ctems, link_rads)
free_boxes = get_reachable_cells(Nwdh, L_CELL, reach_center_dict, MAX_REACH_DICT)
gtimer.toc("extract_available")

# draw_cells(graph, "col_cell", coll_boxes, L_CELL, color=(0.9, 0, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "col_cell")
# draw_cells(graph, "reachable", free_boxes, L_CELL, color=(0.2, 0.7, 0.2, 0.1))
# remove_geometries_by_prefix(graph, "reachable")

2.132892608642578

In [20]:
# draw_cells(graph, "test", [[8,8,8]], L_CELL, color=(1, 1, 0.0, 0.3))

# LEVEL: SCENE

## sample objects

In [21]:
obj_dat = []
for geo_gen in OBJ_GEN_LIST:
    obj_boxes = random.sample(free_boxes,No)
#     draw_cells(graph, "obj_cell", obj_boxes, L_CELL, color=(0.2, 0.2, 0.7, 0.1))
#     remove_geometries_by_prefix(graph, "obj_cell")
    for nbox in obj_boxes:
        gtype, dims, color = geo_gen(L_MAX)
#         if gtype.name == "BOX":
#             nbox_bak = nbox
#         else:
#             nbox = nbox_bak
        obj_dat.append({"nbox": nbox, "gtype": gtype.name, "dims": dims, "color": color, 
                        "center": tuple(np.multiply(nbox, L_CELL)+np.random.random((3,))*L_MAX+0.5*L_CELL - 0.5*L_MAX), 
                        "rpy": np.random.random((3,))*np.pi*2})

## exclude colliding object

In [22]:
__obj_dat=[]
coll_list = []
for odat in obj_dat:
    gtype, center, rpy, dims = getattr(GEOTYPE,odat['gtype']), odat['center'], odat['rpy'], odat['dims']
    ctem = getPointList(get_vertex_rows(gtype, center, rpy, dims))
    crad = (dims[0]/2) if gtype in [GEOTYPE.CAPSULE, GEOTYPE.CYLINDER, GEOTYPE.SPHERE] else 0
    if any(np.array(get_distance_list(ctem, link_ctems, crad, link_rads))<LINK_COLL_MARGIN):
        odat['color'] = (0.9, 0.0, 0.0, 0.3)
        coll_list.append(odat)
    else:
        __obj_dat.append(odat)
SAMPLED_DATA["SCENE"]["obj_dat"] = obj_dat = __obj_dat

In [23]:
SCENE_PATH = os.path.join(WORLD_PATH, "SCENE-"+get_now())
os.mkdir(SCENE_PATH)
save_json(os.path.join(SCENE_PATH, SCENE_FILENAME), SAMPLED_DATA["SCENE"])


## put objects

In [24]:
if "obj_list" in locals(): 
    for obj in obj_list: graph.remove_geometry(obj)
obj_list=[]
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name,*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

In [25]:
if "col_obj_list" in locals(): 
    for obj in col_obj_list: graph.remove_geometry(obj)
col_obj_list=[]
for odat in coll_list:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="col_obj_{}_{}_{}".format(*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=False, fixed=True)
    col_obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

In [26]:
if "col_obj_list" in locals(): 
    for obj in col_obj_list: graph.remove_geometry(obj)

In [27]:
if VISUALIZE: graph.set_rviz()

# LEVEL: ACTION

## DataCollector

In [28]:
class DataCollector:
    def __init__(self, graph, GRIPPER_REFS, S_F_RATIO=2.0):
        self.manager = PriorityQueueManager()
        self.manager.start()
        self.dict_lock = self.manager.Lock()
        self.graph = graph
        self.ghnd = graph.ghnd
        self.GRIPPER_REFS = GRIPPER_REFS
        self.S_F_RATIO = S_F_RATIO
        
    def pick_search(self, ID, obj_list, Q_s, mplan, L_CELL, timeout=1, N_search=100, N_retry=5):
        graph, GRIPPER_REFS = self.graph, self.GRIPPER_REFS
        fail_count = 0
        succ_count = 0
        for i in range(N_search):
            rname, inhand, obj, _, dims_bak, color_bak= sample_pick(GRIPPER_REFS, obj_list, L_CELL, self.ghnd)
            for _ in range(N_retry):
                trajectory, Q_last, error, success = test_pick(graph, GRIPPER_REFS, rname, inhand, obj, None, Q_s, mplan, timeout=timeout)
                if success: break
            print("{}: {} - {}".format(ID, "SUCCESS" if success else "FAILURE", i))
            if success or fail_count < succ_count*self.S_F_RATIO:
                self.dict_lock.acquire()
                idx = self.snode_counter.value
                self.snode_dict[idx] = {
                    "rname1": rname, "obj1": gtem_to_dict(inhand),
                    "obj2": gtem_to_dict(obj), "rname2": None, "dims_bak": dims_bak, "color_bak": color_bak,
                    "success": success, "trajectory": trajectory}
                self.snode_counter.value = self.snode_counter.value+1
                self.dict_lock.release()
                if success:
                    succ_count+=1
                else:
                    fail_count+=1
                print("=========== {} {} {} =========== - {}".format(rname, ID, "SUCCESS" if success else "FAILURE", idx))
            reset_rendering(graph, "PICK", [obj], [inhand], dims_bak, color_bak, vis=False)
        print("=============== TERMINATE {} ==============".format(ID))
        
    def place_search(self, ID, obj_list, Q_s, mplan, L_CELL, timeout=1, N_search=100, N_retry=1):
        graph, GRIPPER_REFS = self.graph, self.GRIPPER_REFS
        fail_count = 0
        succ_count = 0
        for i in range(N_search):
            rname, inhand, ontarget, _, dims_bak, color_bak= sample_place(GRIPPER_REFS, obj_list, L_CELL, self.ghnd)
            for _ in range(N_retry):
                trajectory, Q_last, error, success = test_place(graph, GRIPPER_REFS, rname, inhand, ontarget, None, Q_s, mplan, timeout=timeout)
                if success: break
            print("{}: {} - {}".format(ID, "SUCCESS" if success else "FAILURE", i))
            if success or fail_count < succ_count*self.S_F_RATIO:
                self.dict_lock.acquire()
                idx = self.snode_counter.value
                self.snode_dict[idx] = {
                    "rname1": rname, "obj1": gtem_to_dict(inhand),
                    "obj2": gtem_to_dict(ontarget), "rname2": None, "dims_bak": dims_bak, "color_bak": color_bak,
                    "success": success, "trajectory": trajectory}
                self.snode_counter.value = self.snode_counter.value+1
                self.dict_lock.release()
                if success:
                    succ_count+=1
                else:
                    fail_count+=1
                print("=========== {} {} {} =========== - {}".format(rname, ID, "SUCCESS" if success else "FAILURE", idx))
            reset_rendering(graph, "PLACE", [], [ontarget, inhand], dims_bak, color_bak, vis=False)
        print("=============== TERMINATE {} ==============".format(ID))

    def handover_search(self, ID, obj_list, Q_s, mplan_dict, L_CELL, timeout=1, N_search=100, N_retry=1):
        graph, GRIPPER_REFS = self.graph, self.GRIPPER_REFS
        fail_count = 0
        succ_count = 0
        for i in range(N_search):
            src, tar = random.sample(GRIPPER_REFS.items(),2)
            mplan = mplan_dict[(src[0],tar[0])]
            src, handed, intar, tar, dims_bak, color_bak= sample_handover(src, tar, L_CELL, mplan.ghnd)
            for _ in range(N_retry):
                trajectory, Q_last, error, success = test_handover(graph, GRIPPER_REFS, src, handed, intar, tar, Q_s, mplan, timeout=timeout)
                if success: break
            print("{}: {} - {}".format(ID, "SUCCESS" if success else "FAILURE", i))
            if success or fail_count < succ_count*self.S_F_RATIO:
                self.dict_lock.acquire()
                idx = self.snode_counter.value
                self.snode_dict[idx] = {
                    "rname1": src, "obj1": gtem_to_dict(handed),
                    "obj2": gtem_to_dict(intar), "rname2": tar, "dims_bak": dims_bak, "color_bak": color_bak,
                    "success": success, "trajectory": trajectory}
                self.snode_counter.value = self.snode_counter.value+1
                self.dict_lock.release()
                if success:
                    succ_count+=1
                else:
                    fail_count+=1
                print("=========== {}-{} {} {} =========== - {}".format(src, tar, ID, "SUCCESS" if success else "FAILURE", idx))
            reset_rendering(graph, "HANDOVER", [], [handed, intar], dims_bak, color_bak, vis=False)
        print("=============== TERMINATE {} ==============".format(ID))

    @record_time
    def search_loop_mp(self, Q_s, obj_list, mplan, search_fun, L_CELL, N_agents=None, timeout=1, N_search=100, N_retry=1):
        gtimer.tic("search_loop_mp")
        if N_agents is None:
            N_agents = cpu_count()
        self.N_agents = N_agents
        print("Use {}/{} agents".format(N_agents, cpu_count()))
        self.snode_counter = self.manager.Value('i', 0)
        self.snode_dict = self.manager.dict()
        self.proc_list = [Process(
            target=search_fun,
            args=(id_agent, obj_list, Q_s, mplan, L_CELL),
            kwargs={'timeout':timeout, 'N_search': N_search, 'N_retry': N_retry}) 
                          for id_agent in range(N_agents)]
        for proc in self.proc_list:
            proc.start()

        for proc in self.proc_list:
            proc.join()
        t = gtimer.toc("search_loop_mp")
        print("================== FINISHED in {} sec. ( {} / {} ) =======================".format(round(t/1000,1), self.snode_counter.value, N_agents*N_search))
        print(self.snode_counter.value)
        
    def play_all(self, graph, GRIPPER_REFS, key, test_fun, Q_s, period=0.05, remove_map=[[1],[0]]):
        for k in range(self.snode_counter.value):
            rname, inhand, obj, tar, dims_bak, color_bak, succ, trajectory = load_manipulation_from_dict(self.snode_dict[k], graph.ghnd)
            if succ:
                show_manip_coords(graph, GRIPPER_REFS, key, rname, inhand, obj, rname2=tar)
                graph.show_motion(trajectory, period=period)
            print("DONE: {}".format(k))
            remove1 = [[inhand, obj][iii] for iii in remove_map[0]]
            remove2 = [[inhand, obj][iii] for iii in remove_map[1]]
            reset_rendering(graph, key, remove1, remove2, dims_bak, color_bak, sleep=True, vis=True)

In [29]:
dcol = DataCollector(graph, GRIPPER_REFS)

In [30]:
if VISUALIZE: graph.set_rviz()

## handover case - make in-hand object

In [31]:
dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path, graph.link_names, graph.combined_robot.robot_names)

In [32]:
dcol.search_loop_mp(Q_s, obj_list, dual_mplan_dict, search_fun=dcol.handover_search, L_CELL=L_CELL, N_agents=None, N_search=20, N_retry=1)

Use 20/20 agents
0: FAILURE - 0
1: FAILURE - 0
3: FAILURE - 0
5: FAILURE - 0
2: FAILURE - 0
4: FAILURE - 0
7: FAILURE - 0
6: FAILURE - 0
11: FAILURE - 0
8: FAILURE - 0
9: FAILURE - 0
10: FAILURE - 0
12: FAILURE - 0
16: FAILURE - 0
13: FAILURE - 0
15: FAILURE - 0
18: FAILURE - 0
14: FAILURE - 0
19: FAILURE - 0
17: FAILURE - 0
0: FAILURE - 1
1: FAILURE - 1
5: FAILURE - 1
6: FAILURE - 1
2: FAILURE - 1
12: FAILURE - 1
4: FAILURE - 1
8: FAILURE - 1
9: FAILURE - 1
7: FAILURE - 1
11: FAILURE - 1
3: FAILURE - 1
18: FAILURE - 1
10: FAILURE - 1
13: FAILURE - 1
16: FAILURE - 1
15: FAILURE - 1
14: FAILURE - 1
19: FAILURE - 1
17: FAILURE - 1
0: FAILURE - 2
1: FAILURE - 2
6: FAILURE - 2
5: FAILURE - 2
12: FAILURE - 2
4: FAILURE - 2
8: FAILURE - 2
2: FAILURE - 2
10: FAILURE - 2
11: FAILURE - 2
9: FAILURE - 2
7: FAILURE - 2
3: FAILURE - 2
18: FAILURE - 2
13: FAILURE - 2
16: FAILURE - 2
15: FAILURE - 2
19: FAILURE - 2
17: FAILURE - 2
14: FAILURE - 2
1: FAILURE - 3
0: FAILURE - 3
6: FAILURE - 3
4: FAILU

3: FAILURE - 25
9: FAILURE - 25
4: FAILURE - 25
17: FAILURE - 25
11: FAILURE - 25
18: FAILURE - 25
14: FAILURE - 25
19: FAILURE - 25
16: FAILURE - 25
10: FAILURE - 25
13: FAILURE - 25
12: FAILURE - 25
8: FAILURE - 26
0: FAILURE - 26
1: FAILURE - 26
2: FAILURE - 26
15: FAILURE - 26
5: FAILURE - 26
18: FAILURE - 26
9: FAILURE - 26
3: FAILURE - 26
6: FAILURE - 26
14: FAILURE - 26
19: FAILURE - 26
7: FAILURE - 26
11: FAILURE - 26
4: FAILURE - 26
17: FAILURE - 26
10: FAILURE - 26
13: FAILURE - 26
12: FAILURE - 26
16: FAILURE - 26
0: FAILURE - 27
8: FAILURE - 27
1: FAILURE - 27
5: FAILURE - 27
2: FAILURE - 27
9: FAILURE - 27
18: FAILURE - 27
15: FAILURE - 27
14: FAILURE - 27
6: FAILURE - 27
3: FAILURE - 27
7: FAILURE - 27
19: FAILURE - 27
11: FAILURE - 27
17: FAILURE - 27
4: FAILURE - 27
10: FAILURE - 27
13: FAILURE - 27
16: FAILURE - 27
12: FAILURE - 27
0: FAILURE - 28
8: FAILURE - 28
1: FAILURE - 28
2: FAILURE - 28
5: FAILURE - 28
18: FAILURE - 28
6: FAILURE - 28
9: FAILURE - 28
15: FAILUR

6: FAILURE - 49
=============== TERMINATE 11 ==============
=============== TERMINATE 18 ==============
=============== TERMINATE 6 ==============
3: FAILURE - 49
=============== TERMINATE 3 ==============
9: FAILURE - 49
=============== TERMINATE 9 ==============
16: FAILURE - 49
0: FAILURE - 49
=============== TERMINATE 0 ==============
=============== TERMINATE 16 ==============
1: FAILURE - 49
=============== TERMINATE 1 ==============
19: FAILURE - 49
=============== TERMINATE 19 ==============
14: FAILURE - 49
=============== TERMINATE 14 ==============
15: FAILURE - 49
17: FAILURE - 49
=============== TERMINATE 15 ==============
=============== TERMINATE 17 ==============
2: FAILURE - 49
=============== TERMINATE 2 ==============
5: FAILURE - 49
=============== TERMINATE 5 ==============
7: FAILURE - 49
=============== TERMINATE 7 ==============
13: FAILURE - 49
=============== TERMINATE 13 ==============
10: FAILURE - 49
12: FAILURE - 49
=============== TERMINATE 10 ===========

In [33]:
save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})

## Playall

In [34]:
if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "HANDOVER", test_pick, Q_s, remove_map=[[],[0,1]])

# moveit

In [35]:
mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names, binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
graph.set_planner(mplan)
mplan.update(graph)

## pick case

In [36]:
gtimer.reset()
dcol.search_loop_mp(Q_s, obj_list, mplan, search_fun=dcol.pick_search, L_CELL=L_CELL, N_agents=None, timeout=1, N_search=20, N_retry=1)

Use 20/20 agents
18: SUCCESS - 0
=========== panda1 18 SUCCESS =========== - 0
2: FAILURE - 0
0: FAILURE - 0
1: FAILURE - 0
4: FAILURE - 0
5: FAILURE - 0
3: FAILURE - 0
7: FAILURE - 0
6: FAILURE - 0
8: FAILURE - 0
9: FAILURE - 0
10: FAILURE - 0
12: FAILURE - 0
13: FAILURE - 0
11: FAILURE - 0
14: FAILURE - 0
16: FAILURE - 0
15: FAILURE - 0
17: FAILURE - 0
19: FAILURE - 0
18: FAILURE - 1
=========== indy0 18 FAILURE =========== - 1
4: SUCCESS - 1
=========== indy0 4 SUCCESS =========== - 2
0: FAILURE - 1
2: FAILURE - 1
7: FAILURE - 1
1: FAILURE - 1
3: FAILURE - 1
8: FAILURE - 1
5: FAILURE - 1
6: FAILURE - 1
9: FAILURE - 1
10: FAILURE - 1
12: FAILURE - 1
11: FAILURE - 1
13: FAILURE - 1
14: FAILURE - 1
16: FAILURE - 1
17: FAILURE - 1
19: FAILURE - 1
15: FAILURE - 1
4: SUCCESS - 2
=========== indy0 4 SUCCESS =========== - 3
18: FAILURE - 2
=========== indy0 18 FAILURE =========== - 4
0: FAILURE - 2
2: FAILURE - 2
7: FAILURE - 2
1: FAILURE - 2
6: FAILURE - 2
8: FAILURE - 2
3: FAILURE - 2
5: 

4: FAILURE - 19
=============== TERMINATE 4 ==============
2: FAILURE - 19
=============== TERMINATE 2 ==============
13: FAILURE - 19
=============== TERMINATE 13 ==============
0: FAILURE - 19
=============== TERMINATE 0 ==============
6: FAILURE - 19
=============== TERMINATE 6 ==============
5: FAILURE - 19
=========== panda1 5 FAILURE =========== - 41
=============== TERMINATE 5 ==============
16: FAILURE - 19
=============== TERMINATE 16 ==============
9: FAILURE - 19
=============== TERMINATE 9 ==============
11: FAILURE - 19
=============== TERMINATE 11 ==============
3: FAILURE - 19
=============== TERMINATE 3 ==============
7: FAILURE - 19
=============== TERMINATE 7 ==============
10: FAILURE - 19
=============== TERMINATE 10 ==============
14: FAILURE - 19
=============== TERMINATE 14 ==============
19: FAILURE - 19
=============== TERMINATE 19 ==============
17: SUCCESS - 19
=========== indy0 17 SUCCESS =========== - 42
=============== TERMINATE 17 ==============
15: FAILU

In [37]:
save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})

## Playall

In [38]:
if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "PICK", test_pick, Q_s, remove_map=[[1],[0]])

## place case

In [39]:
gtimer.reset()
dcol.search_loop_mp(Q_s, obj_list, mplan, search_fun=dcol.place_search, L_CELL=L_CELL, N_agents=None, timeout=1, N_search=20, N_retry=1)

Use 20/20 agents
0: FAILURE - 0
1: FAILURE - 0
3: FAILURE - 0
2: FAILURE - 0
4: FAILURE - 0
7: FAILURE - 0
6: FAILURE - 0
5: FAILURE - 0
8: FAILURE - 0
9: FAILURE - 0
11: FAILURE - 0
10: FAILURE - 0
13: FAILURE - 0
12: FAILURE - 0
15: FAILURE - 0
14: FAILURE - 0
17: FAILURE - 0
16: FAILURE - 0
18: FAILURE - 0
19: FAILURE - 0
18: SUCCESS - 1
=========== indy0 18 SUCCESS =========== - 0
1: FAILURE - 1
3: FAILURE - 1
4: FAILURE - 1
2: FAILURE - 1
0: FAILURE - 1
6: FAILURE - 1
5: FAILURE - 1
10: FAILURE - 1
11: FAILURE - 1
9: FAILURE - 1
8: FAILURE - 1
13: FAILURE - 1
7: FAILURE - 1
15: FAILURE - 1
12: FAILURE - 1
14: FAILURE - 1
16: FAILURE - 1
17: FAILURE - 1
19: FAILURE - 1
18: FAILURE - 2
=========== panda1 18 FAILURE =========== - 1
2: FAILURE - 2
1: FAILURE - 2
3: FAILURE - 2
0: FAILURE - 2
4: FAILURE - 2
5: FAILURE - 2
11: FAILURE - 2
6: FAILURE - 2
7: FAILURE - 2
10: FAILURE - 2
8: FAILURE - 2
9: FAILURE - 2
13: FAILURE - 2
15: FAILURE - 2
12: FAILURE - 2
17: FAILURE - 2
14: FAILUR

=============== TERMINATE 10 ==============
15: FAILURE - 19
=============== TERMINATE 15 ==============
17: FAILURE - 19
=============== TERMINATE 17 ==============
16: FAILURE - 19
=============== TERMINATE 16 ==============
================== FINISHED in 23.2 sec. ( 25 / 400 ) =======================
25


In [40]:
save_json(os.path.join(SCENE_PATH, get_now()+".json"),  {idx: {k:v for k,v in item.items() if k !="trajectory"} for idx, item in dcol.snode_dict.items()})

## Playall

In [41]:
if VISUALIZE: dcol.play_all(graph, GRIPPER_REFS, "PLACE", test_place, Q_s, remove_map=[[],[0,1]])

In [42]:
if VISUALIZE: graph.set_rviz()